<a href="https://colab.research.google.com/github/mahdiSheykhiGithub/Time-Series-Forecasting/blob/main/climate_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Climate/climate.csv', index_col='Date Time')
data

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2


# Preprocessing


In [ ]:
data.index = pd.to_datetime(data.index, format='%d.%m.%Y %H:%M:%S')

In [ ]:
data = data[(data.index.minute == 0) & (data.index.hour % 3 == 0)]
data

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 03:00:00,996.84,-8.81,264.59,-9.66,93.50,3.13,2.93,0.20,1.83,2.94,1312.18,0.18,0.63,167.2
2009-01-01 06:00:00,997.71,-9.67,263.66,-10.62,92.70,2.93,2.71,0.21,1.69,2.72,1317.71,0.05,0.50,146.0
2009-01-01 09:00:00,999.69,-7.66,265.52,-8.84,91.20,3.43,3.13,0.30,1.95,3.13,1310.14,0.34,0.63,202.2
2009-01-01 12:00:00,1000.30,-6.87,266.27,-8.28,89.60,3.64,3.27,0.38,2.03,3.26,1306.98,1.84,2.63,184.4
2009-01-01 15:00:00,999.88,-5.69,267.48,-7.00,90.40,3.99,3.61,0.38,2.25,3.61,1300.51,1.17,1.88,134.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 12:00:00,1004.76,-1.19,271.60,-5.90,70.20,5.60,3.93,1.67,2.44,3.91,1285.09,0.86,1.56,211.2
2016-12-31 15:00:00,1003.12,5.09,278.00,-3.09,55.34,8.78,4.86,3.92,3.02,4.85,1253.58,0.30,0.70,134.1
2016-12-31 18:00:00,1002.27,-0.08,272.90,-4.90,69.81,6.07,4.24,1.83,2.64,4.23,1276.52,0.39,1.04,220.7


In [ ]:
tempc = data['T (degC)']
tempc

Date Time
2009-01-01 03:00:00   -8.81
2009-01-01 06:00:00   -9.67
2009-01-01 09:00:00   -7.66
2009-01-01 12:00:00   -6.87
2009-01-01 15:00:00   -5.69
                       ... 
2016-12-31 12:00:00   -1.19
2016-12-31 15:00:00    5.09
2016-12-31 18:00:00   -0.08
2016-12-31 21:00:00   -3.09
2017-01-01 00:00:00   -4.82
Name: T (degC), Length: 23363, dtype: float64

In [ ]:
var = data.drop(['T (degC)', 'Tpot (K)'], axis=1)
var

,p (mbar),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,
2009-01-01 03:00:00,996.84,-9.66,93.50,3.13,2.93,0.20,1.83,2.94,1312.18,0.18,0.63,167.2
2009-01-01 06:00:00,997.71,-10.62,92.70,2.93,2.71,0.21,1.69,2.72,1317.71,0.05,0.50,146.0
2009-01-01 09:00:00,999.69,-8.84,91.20,3.43,3.13,0.30,1.95,3.13,1310.14,0.34,0.63,202.2
2009-01-01 12:00:00,1000.30,-8.28,89.60,3.64,3.27,0.38,2.03,3.26,1306.98,1.84,2.63,184.4
2009-01-01 15:00:00,999.88,-7.00,90.40,3.99,3.61,0.38,2.25,3.61,1300.51,1.17,1.88,134.9
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-31 12:00:00,1004.76,-5.90,70.20,5.60,3.93,1.67,2.44,3.91,1285.09,0.86,1.56,211.2
2016-12-31 15:00:00,1003.12,-3.09,55.34,8.78,4.86,3.92,3.02,4.85,1253.58,0.30,0.70,134.1
2016-12-31 18:00:00,1002.27,-4.90,69.81,6.07,4.24,1.83,2.64,4.23,1276.52,0.39,1.04,220.7


In [ ]:
scaler = StandardScaler()
var = scaler.fit_transform(var)
var

array([[ 0.91270836, -2.17133967,  1.06222553, ..., -0.02326059,
        -0.02211692, -0.09183425],
       [ 1.01686391, -2.31396936,  1.0137245 , ..., -0.0252469 ,
        -0.02352123, -0.33721218],
       [ 1.25390756, -2.04951014,  0.92278507, ..., -0.0208159 ,
        -0.02211692,  0.31327082],
       ...,
       [ 1.56278262, -1.46413412, -0.37401116, ..., -0.02005194,
        -0.01768794,  0.52739779],
       [ 1.44545799, -1.7895081 , -0.13817491, ..., -0.01669049,
        -0.0129349 ,  0.3746153 ],
       [ 1.26947103, -1.98710965, -0.01692234, ..., -0.00721732,
        -0.00774976,  0.11303317]])

In [ ]:
seq_len = 56
step = 2

sequences = []
targets = []

for i in range(0, len(data)-seq_len-7, step):
    seq = var[i:i+seq_len]
    target = tempc[i+seq_len+7]
    sequences.append(seq)
    targets.append(target)

sequences, targets = np.array(sequences), np.array(targets)

In [ ]:
sequences.shape

(11650, 56, 12)

In [ ]:
X_train, X_val, X_test = sequences[:8000], sequences[8000:9500], sequences[9500:]
y_train, y_val, y_test = targets[:8000], targets[8000:9500], targets[9500:]

# GRU Modeling

In [ ]:
model = Sequential()
model.add(GRU(units=256, activation='tanh', recurrent_dropout=0.3, input_shape=(56,12), return_sequences=True))
model.add(GRU(units=256, activation='tanh', recurrent_dropout=0.3, dropout=0.3))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 56, 256)           207360    
                                                                 
 gru_3 (GRU)                 (None, 256)               394752    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 635137 (2.42 MB)
Trainable params: 63513

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.001)
loss = tf.losses.MeanAbsoluteError()
model.compile(optimizer=opt, loss=loss)

In [ ]:
model.fit(X_train, y_train, batch_size=50, epochs=24, validation_data=(X_val, y_val), verbose=2)

Epoch 1/24
160/160 - 30s - loss: 4.1289 - val_loss: 2.6878 - 30s/epoch - 190ms/step
Epoch 2/24
160/160 - 26s - loss: 3.0761 - val_loss: 2.6244 - 26s/epoch - 165ms/step
Epoch 3/24
160/160 - 27s - loss: 2.9740 - val_loss: 2.5166 - 27s/epoch - 166ms/step
Epoch 4/24
160/160 - 26s - loss: 2.9219 - val_loss: 2.4456 - 26s/epoch - 164ms/step
Epoch 5/24
160/160 - 26s - loss: 2.9079 - val_loss: 2.4367 - 26s/epoch - 165ms/step
Epoch 6/24
160/160 - 26s - loss: 2.8775 - val_loss: 2.3969 - 26s/epoch - 165ms/step
Epoch 7/24
160/160 - 26s - loss: 2.8752 - val_loss: 2.4119 - 26s/epoch - 164ms/step
Epoch 8/24
160/160 - 26s - loss: 2.8615 - val_loss: 2.3691 - 26s/epoch - 163ms/step
Epoch 9/24
160/160 - 26s - loss: 2.8390 - val_loss: 2.4012 - 26s/epoch - 164ms/step
Epoch 10/24
160/160 - 26s - loss: 2.8106 - val_loss: 2.4339 - 26s/epoch - 164ms/step
Epoch 11/24
160/160 - 26s - loss: 2.8234 - val_loss: 2.3610 - 26s/epoch - 163ms/step
Epoch 12/24
160/160 - 26s - loss: 2.7989 - val_loss: 2.3917 - 26s/epoch - 